In [1]:
import os
import fire
import matplotlib.pyplot as plt
import torch
import torch.optim.swa_utils as swa_utils
try:
    import torchcde
except ImportError as e:
    raise ImportError("`torchcde` is not installed: go to https://github.com/patrick-kidger/torchcde.") from e
import torchsde
import tqdm
import numpy as np

from deep_fields import data_path
from deep_fields.data.simulations.sdes import get_data
from deep_fields.models.crypto.predictors import CryptoSeq2Seq, CryptoSeq2NeuralSDE
from deep_fields.data.crypto.dataloaders import CryptoDataLoader
from deep_fields.models.sdes.neural_sde import Generator, Discriminator
from deep_fields.models.utils.utils_ts import unfold_steps_ahead
device = torch.device("cpu")

# Encoder Part

In [2]:
crypto_folder = os.path.join(data_path, "raw", "crypto")
data_folder = os.path.join(crypto_folder, "2021-06-14")
kwargs = {"path_to_data":data_folder,
          "batch_size": 29}
data_loader = CryptoDataLoader('cpu', **kwargs)
data_batch = next(data_loader.train.__iter__())

# Neural SDE Decoder

In [3]:
model_param = CryptoSeq2NeuralSDE.get_parameters()
inference_param = CryptoSeq2NeuralSDE.get_inference_parameters()

cs2n = CryptoSeq2NeuralSDE(**model_param)
cs2n.initialize_inference(data_loader, parameters=None, **inference_param)

New Model Set For Inference
(<class 'AttributeError'>, AttributeError("'NoneType' object has no attribute 'train'"), <traceback object at 0x000002EC8CEFE3C0>)
Optimizer Not Created In Abstract Initialize Inference


In [4]:
encoder_forward,generated_samples = cs2n.generate(data_batch)

In [5]:
generated_samples.shape

torch.Size([68295, 4, 4])